# TD2 part 2: Named entity recognition

Dans ce TD, nous allons prendre un datasets où les noms de personnes sont taggés.<br>
Nous allons transformer ces données en tenseurs X, y et attention_mask.<br>
Nous allons créer un modèle RNN pour prédire si un mot est un nom de personne.<br>
Nous allons ensuite créer la loop avec l'optimizer pour apprendre le modèle.<br>
Du modèle appris (prédisant sur les tokens), nous allons postprocess les prédictions pour avoir les prédictions sur les noms.

Un fois que la loop est créée et que le modèle apprend, nous allons changer la structure du modèle:
- Changer learning rate. Comment se comporte le modèle
- Ajouter des couches denses, ReLU, dropout, normalization
- Changer le nombre de layers du RNN, LSTM.

Lorsqu'on a un bon modèle de prédiction pour les noms de personnes, nous allons l'appliquer à notre projet fil rouge.
Utilisez-le tel que. Quelle accuracy ?
Ré-entrainez la (les) dernière(s) couche(s) du modèle sur notre jeu de données. A-t-il gagné en accuracy ?

In [1]:
# Import
import matplotlib.pyplot as plt
import numpy as np
import torch
import transformers

## Data

Télécharger le dataset MultiNERD FR [ici](https://github.com/Babelscape/multinerd)<br>
Mettez les données dans le dossier data/raw du projet.


In [2]:
def extract_multinerd_person_words(filename="../data/raw/train_fr.tsv"):
    with open(filename, encoding='utf-8') as f:
        tagged_words = [line.strip().split("\t") for line in f]
        
        # Joining words until we meet a dot
        # Word's label is 1 if 'PER' is in its tag
        sentences = []
        sentence_labels = []
    
        this_word = []
        this_labels = []
        for tagged_word in tagged_words:
            if len(tagged_word) < 3:
                # not a tagged word
                continue
            word = tagged_word[1]
            tag = tagged_word[2]
        
            if word == '.':
                sentences.append(" ".join(this_word))
                sentence_labels.append(this_labels)
            
                this_word = []
                this_labels = []
            else:
                this_word.append(word)
                this_labels.append(1 * tag.endswith("PER"))

    return sentences, sentence_labels


In [3]:
sentences, labels = extract_multinerd_person_words("../data/raw/multinerd_train.tsv")

In [4]:
print(labels[1])
print(sentences[1])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c’ est ainsi que des firmes comme DuPont , Dow Chemical , Monsanto , American Cyanamid lancèrent la production en masse d' engrais minéraux


## Tokenizer

En utilisant le tokenizer d'HuggingFace "camembert-base":
- Transformer les phrases en tokens
- Obtenez des vecteur y qui ont le même nombre d'entrées qu'il y a de tokens dans la phrase
- Ayez un tenseur "attention_mask" pour savoir sur quels tokens on cherche à predire le label
- Transformez les tokens en token_ids (avec le tokenizer)
Avec tout cela, vous pouvez former vos tenseurs X, Y et attention_mask

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("camembert-base")

In [6]:
def build_tokens_and_labels_and_attention_mask(tokenizer, sentence, labels):
    words = sentence.split()
    tokens = []
    tokens_label = []
    attention_mask = []
    
    for word, label in zip(words, labels):
        this_tokens = tokenizer.tokenize(word)
        tokens += this_tokens
        
        this_labels = [0] * len(this_tokens)
        this_labels[0] = label        
        tokens_label += this_labels
        
        this_attention_mask = [1] + [0] * (len(this_tokens) - 1)
        attention_mask += this_attention_mask
        
    return tokens, tokens_label, attention_mask

In [7]:
tokens, label, padding_masks = build_tokens_and_labels_and_attention_mask(tokenizer, sentences[0], labels[0])

In [8]:
print(tokens)
print(label)
print(padding_masks)

['▁Il', '▁est', '▁incarné', '▁par', '▁A', 'ustin', '▁S', 'to', 'well']
[0, 0, 0, 0, 1, 0, 1, 0, 0]
[1, 1, 1, 1, 1, 0, 1, 0, 0]


In [9]:
X = []
Y = []
attention_masks = []
for sentence, label in zip(sentences, labels):
    tokens, label, padding_masks = build_tokens_and_labels_and_attention_mask(tokenizer, sentence, label)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    X.append(token_ids)
    Y.append(label)
    attention_masks.append(padding_masks)

In [14]:
print(f'{len(attention_masks[0])} et {len(X[0])} et {len(Y[0])}')

9 et 9 et 9


## Model

Contruisez un modèle RNN comme dans la partie 1. Pour l'instant, il prendra comme arguments:
- Vocab size: le nombre de différents tokens du tokenizer (52 000 pour camembert-base)
- Embedding dim: la dimension de l'embedding des tokens (par défaut 50)
- hidden_dim: la dimension de l'état récurrent de votre RNN (par défaut 20)
- tagset_size: la nombre de classes possibles pour les prédictions (ici 2)

Dans le forward, votre modèle enchaînera les couches suivantes:
- Un embedding
- Un RNN
- Un ReLU
- Une couche linéaire
- Un softmax pour que la somme des prédictions pour une entrée soit égale à 1 (la prédiction pour un élément et sa probabilité d'être dans chaque classe)

In [15]:
import torch.nn as nn
import torch.nn.functional as F
class RNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim = 50, hidden_dim = 20, target_size = 2):
        super(RNNModel,self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, target_size)

    def forward(self, x):
        embd = self.embedding(x)
        out, _ = self.rnn(embd)
        out = F.relu(out)
        out = self.linear(out)
        out = F.log_softmax(out, dim=-1)
        return out


## Optimizer

Je fournis ici une fonction prenant un modèle, des tenseurs X, y, attention_mask.
Pour chaque batch:
- La loop utilise le modèle pour prédire sur x_batch
- Avec attention_mask, elle identifie sur quels tokens les prédictions compte
- Elle regarde la cross entropy entre y\[attention_ids\] et yhat\[attention_ids\]
- Elle output un dictionnaire avec le model et la loss au fur et à mesure des itérations

Entraînez le modèle avec vos données. <br>
Plottez la loss history.<br>
Itérez sur le modèle pour l'améliorer:
- Changer learning rate. Comment se comporte le modèle
- Ajouter des couches denses, ReLU, dropout, normalization
- Changer le nombre de layers du RNN, LSTM.



In [16]:
model = RNNModel(vocab_size=52000, embedding_dim=50, hidden_dim=20, target_size=2)
def train_model(model, X, y, attention_masks, n_epochs=100, lr=0.05, batch_size=128):
    loss_function = torch.nn.CrossEntropyLoss()
    loss_history = []

    dataset = torch.utils.data.TensorDataset(X, y, attention_masks)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(n_epochs):
        for i, (x_batch, y_batch, mask) in enumerate(loader):
            optimizer.zero_grad()
        
            ids = mask.reshape(-1)
            yhat = model(x_batch).reshape((-1, 2))[ids]
            this_y = y_batch.reshape(-1)[ids]
            
            loss = loss_function(yhat, this_y)
            loss.backward()
            
            loss_history.append(loss.clone().detach())
        
            optimizer.step()
        if epoch % 10 == 0:
            print(f"Got loss at {epoch}", np.mean(loss_history[-10:]))
    
    return {"model": model, "loss_history": loss_history}

In [17]:
from torch.nn.utils.rnn import pad_sequence
X_tensors = [torch.tensor(seq, dtype=torch.long) for seq in X]
X_padded = pad_sequence(X_tensors, batch_first=True, padding_value=0)

Y_tensors = [torch.tensor(seq, dtype=torch.long) for seq in Y]
Y_padded = pad_sequence(Y_tensors, batch_first=True, padding_value=0)

attention_masks_tensor = [torch.tensor(seq, dtype=torch.long) for seq in attention_masks]
attention_masks_padded = pad_sequence(attention_masks_tensor, batch_first=True, padding_value=0)

train_model(model, X_padded, Y_padded, attention_masks_padded)

Got loss at 0 0.79506063
Got loss at 10 0.004845935
Got loss at 20 2.0942482e-05
Got loss at 30 0.012983801
Got loss at 40 0.0050909407
Got loss at 50 9.8497476e-05
Got loss at 60 0.00018422399
Got loss at 70 5.7559075e-05
Got loss at 80 0.552932
Got loss at 90 0.00044702404


{'model': RNNModel(
   (embedding): Embedding(52000, 50)
   (rnn): RNN(50, 20, batch_first=True)
   (linear): Linear(in_features=20, out_features=2, bias=True)
 ),
 'loss_history': [tensor(0.7551),
  tensor(0.6089),
  tensor(0.3157),
  tensor(0.2880),
  tensor(0.1850),
  tensor(0.0604),
  tensor(0.1116),
  tensor(0.0070),
  tensor(0.0237),
  tensor(0.0094),
  tensor(0.0045),
  tensor(0.0035),
  tensor(0.0011),
  tensor(0.0009),
  tensor(0.0071),
  tensor(0.0004),
  tensor(0.0005),
  tensor(0.0010),
  tensor(3.0840e-05),
  tensor(0.0008),
  tensor(0.0017),
  tensor(0.0001),
  tensor(0.0002),
  tensor(0.0025),
  tensor(0.0004),
  tensor(0.0004),
  tensor(5.5162e-05),
  tensor(7.6584e-05),
  tensor(0.0002),
  tensor(0.0003),
  tensor(0.0001),
  tensor(1.2169e-05),
  tensor(4.1069e-05),
  tensor(0.0003),
  tensor(3.1058e-05),
  tensor(2.1839e-06),
  tensor(0.0002),
  tensor(1.2456e-05),
  tensor(1.1194e-05),
  tensor(5.3687e-05),
  tensor(2.3374e-05),
  tensor(0.0002),
  tensor(1.5688e-05)

## Postprocessing

Créer une fonction prenant les prédictions du modèle (au niveau token) et sort les prédictions au niveau mot.<br>
Par exemple, admettons que, pour un mot, la prédiction du 1er token est la seule qu'on considère.<br>
si la phrase est "Bonjour John", avec les tokens \["bon", "jour", "Jo", "hn"\] avec les predictions \[0.12, 0.65, 0.88, 0.45\]<br>
Je veux récupérer les prédictions "bonjour": 0.12, "John": 0.88

In [18]:
torch.save(model, "model.pth")

In [53]:
def word_level_predictions(tokens, token_predictions):
    word_predictions = {}
    current_word = ""
    current_prediction = 0.0

    for i, token in enumerate(tokens):
        # Si le token est le début d'un nouveau mot ou la continuation d'un mot existant
        if i == 0 or not tokens[i-1].isalpha() or not token.isalpha():
            if current_word:  # S'il y a un mot précédent, l'ajouter aux prédictions
                word_predictions[current_word] = current_prediction
            current_word = token
            current_prediction = token_predictions[i]
        else:  # Si le token est une continuation sans préfixe
            current_word += token

    # Ajouter la prédiction du dernier mot
    if current_word:
        word_predictions[current_word] = current_prediction

    return word_predictions

# Exemple d'utilisation
tokens = ["bon", "jour", "Jo", "hn"]
token_predictions = [0.12, 0.65, 0.88, 0.45]
word_predictions = word_level_predictions(tokens, token_predictions)
print(word_predictions)


{'bonjourJohn': 0.12}
